# Język Python - Wykład 4.

## Context Manager 

Problem - otwieramy plik lub pozyskujemy jakies dane zewnętrzne. Chcemy mieć pewność, że po wykonaniu operacji na pliku / danych, plik zostanie zamknięty, a zasoby zwolnione. Czyli chcemy wykonać:
    
    set things up
    do something
    tear things down

Ostatnie polecenie musi się wykonać niezależnie od powodzenia operacji na żądanych zasobach. 

Rozwiązanie:

    set things up
    try:
        do something
    finally:
        tear things down

Gdy podobne zadania wykonujemy wielokrotnie, wygodnie będzie kontrolę zasobów przenieść do odrębnej funkcji:

    def controlled_execution(callback):
        set things up
        try:
            callback(thing)
        finally:
            tear things down

    def my_function(thing):
        do something

    controlled_execution(my_function)

Ale to nieco rozwlekły, zwłaszcza jeśli trzeba zmodyfikować zmienne lokalne. Innym sposobem jest użycie generatora jednoelementowego, a następnie wykorzystanie go w pętli:

    def controlled_execution():
        set things up
        try:
            yield thing
        finally:
            tear things down

    for thing in controlled_execution():
        do something with thing

Pętla dla czegoś co wykonujemy tylko raz nie jest jednak zbyt eleganckim rozwiązaniem. Dlatego:

    class controlled_execution:
        def __enter__(self):  # context guard
            set things up
            return thing
        def __exit__(self, type, value, traceback):
            tear things down

    with controlled_execution() as thing:
         some code

    with expression [as (targets)]:
        code block with context of "targets"

* Możliwość utworzenia dynamicznego (w runtime) kontekstu dla wykonania sekcji kodu
* Przykłady użycia: 
   * czytanie z pliku - w kontekście otwarcia i zamknięcia pliku
   * zakładanie i zwalnianie blokad ("lock")
   * zmiana i odtworzenie globalnego stanu
* Obiekt context managera musi implementować protokół: ``__enter__()`` i ``__exit__()``
* [PEP 343](http://www.python.org/dev/peps/pep-0343/)

In [18]:
with open('Python2016_Wyklad-04.ipynb') as notebook:
    print(notebook.read(50))
    print(notebook)
print(notebook)

<_io.TextIOWrapper name='Python2016_Wyklad-04.ipynb' mode='r' encoding='cp1251'>
{
 "cells": [
  {
   "cell_type": "markdown",
   "
<_io.TextIOWrapper name='Python2016_Wyklad-04.ipynb' mode='r' encoding='cp1251'>
<_io.TextIOWrapper name='Python2016_Wyklad-04.ipynb' mode='r' encoding='cp1251'>


In [14]:
simple_notebook = open('Python2016_Wyklad-04.ipynb')
print(simple_notebook.read(50))
print(simple_notebook)
simple_notebook.close()
print(simple_notebook)

{
 "cells": [
  {
   "cell_type": "markdown",
   "
<_io.TextIOWrapper name='Python2016_Wyklad-04.ipynb' mode='r' encoding='cp1251'>
<_io.TextIOWrapper name='Python2016_Wyklad-04.ipynb' mode='r' encoding='cp1251'>


In [17]:
class MyContextManager:
    
    def __enter__(self):
        print("Context prepared")
    
    def __exit__(self, exc_type, exc_value, traceback):
        print("Context closed ", exc_type, exc_value, traceback)

        
with MyContextManager() as f:
    print("Hello", f)

Context prepared
Hello None
Context closed  None None None


In [19]:
class MyContextManager():
    
    def __enter__(self):
        print("Context prepared")
    
    def __exit__(self, exc_type, exc_value, traceback):
        print("Context closed ", exc_type, exc_value, traceback)
        #return True

        
with MyContextManager():
    1/0

Context prepared
Context closed  <class 'ZeroDivisionError'> division by zero <traceback object at 0x008D0490>


ZeroDivisionError: division by zero

In [21]:
class MyContextManager:
    
    def __init__(self, custom="Context"):
        self.custom = custom
        
    def __enter__(self):
        print("{} prepared".format(self.custom))
    
    def __exit__(self, exc_type, exc_value, traceback):
        print("{} closed {} {} {}".format(self.custom, exc_type, exc_value, traceback))

        
with MyContextManager('Dog'):
    print("Hau")

Dog prepared
Hau
Dog closed None None None


### Context manager z użyciem dekoratora

In [24]:
from contextlib import contextmanager

@contextmanager
def tag(name):
    print("<%s>" % name,)
    yield  # Tutaj zostanie wywolany kod z wnetrza context managera
    print("</%s>" % name,)

with tag("h1"):
    print("foo",)

print()

with tag("div"):
    print("foo",)


<h1>
foo
</h1>

<div>
foo
</div>


Przykład z tymczasowym folderem (za http://stackoverflow.com/questions/3012488/what-is-the-python-with-statement-designed-for)

In [23]:
from tempfile import mkdtemp
from shutil import rmtree


@contextmanager
def temporary_dir(*args, **kwds):
    name = mkdtemp(*args, **kwds)
    try:
        yield name
    finally:
        rmtree(name)


with temporary_dir(".temp") as dirname:
    print("doing sth with",dirname)

doing sth with C:\Users\JACEK_~1\AppData\Local\Temp\tmpqbw59dp_.temp


## Metody specjalne c.d.

### ``__repr__`` - the “official” string representation of an object

In [25]:
class T:
    pass

print([T(), T(), T()])
print(repr(T()))
print(str(T()))

[<__main__.T object at 0x04F3BA10>, <__main__.T object at 0x04F3B9F0>, <__main__.T object at 0x04F3B810>]


In [26]:
class T:
    def __repr__(self):
        return 'T class'

print([T(), T(), T()])
print(repr(T()))
print(str(T()))

[T class, T class, T class]
T class
T class


### ``__str__`` - the “informal” or nicely printable string representation of an object.

In [27]:
class Car:
    def __init__(self, vendor, model, ccm):
        self.vendor = vendor
        self.model = model
        self.ccm = ccm
        
    def __str__(self):
        return '%s %s (%.1f L)' % (self.vendor, self.model, self.ccm / 1000.0)

print(Car('Porsche', '911', 3600))
print(repr(Car('Porsche', '911', 3600)))
print(str(Car('Porsche', '911', 3600)))

Porsche 911 (3.6 L)
Porsche 911 (3.6 L)


W przypadku, gdy metoda __str__ nie jest zdefiniowana, wywoływana jest metoda __repr__

### ``__hash__`` - (Hash values are integers. They are used to quickly compare dictionary keys during a dictionary lookup)

In [28]:
class T:
    def __init__(self,x,y):
        self.x = x
        self.y = y
    
    def __str__(self):
        return str(self.x)+" "+str(self.y)
        
    def __eq__(self,other):
        print("comparing to" + str(other))
        return self.x == other.x and self.y == other.y
    
    def __hash__(self):
        return hash(self.x)

d = {T(1,x) : x for x in range(100)}
T(1,10) in d

comparing to1 1
comparing to1 2
comparing to1 2
comparing to1 3
comparing to1 3
comparing to1 3
comparing to1 4
comparing to1 4
comparing to1 4
comparing to1 4
comparing to1 5
comparing to1 5
comparing to1 5
comparing to1 5
comparing to1 5
comparing to1 6
comparing to1 6
comparing to1 6
comparing to1 6
comparing to1 6
comparing to1 6
comparing to1 7
comparing to1 7
comparing to1 7
comparing to1 7
comparing to1 7
comparing to1 7
comparing to1 7
comparing to1 7
comparing to1 8
comparing to1 8
comparing to1 8
comparing to1 8
comparing to1 8
comparing to1 8
comparing to1 8
comparing to1 8
comparing to1 8
comparing to1 9
comparing to1 9
comparing to1 9
comparing to1 9
comparing to1 9
comparing to1 9
comparing to1 9
comparing to1 9
comparing to1 9
comparing to1 9
comparing to1 10
comparing to1 10
comparing to1 10
comparing to1 10
comparing to1 10
comparing to1 10
comparing to1 10
comparing to1 10
comparing to1 10
comparing to1 10
comparing to1 10
comparing to1 11
comparing to1 11
comparing t

True

### ``__bool__`` - Called to implement truth value testing and the built-in operation

In [29]:
bool(None)

False

In [30]:
bool(0)

False

In [31]:
bool([1,3,4])

True

In [32]:
bool("")

False

In [33]:
class T:
    def __len__(self):
        return 2
print(bool(T()))

True


In [34]:
class T:
    def __bool__(self):
        print(id(self))
        if id(self) // 10 % 2 :
            return True
        else:
            return False
print(bool(T()))        

11115280
False


### ``__class__`` - klasa, do której należy instancja

In [35]:
x = 5
x.__class__

int

In [36]:
type(x)

int

In [37]:
type(int)

type

### Emulacja typu kontenera (cd..)

In [38]:
class T():
   
    c = [1, 2, 3]

    def __len__(self):
        return len(self.c)
    
    def __contains__(self, x):
        return x in self.c
    
    def __iter__(self):
        for x in self.c:
            yield x
    
t = T()
print(len(t))
print(2 in t)

3
True


In [39]:
iter(t)

<generator object __iter__ at 0x00A67670>

In [40]:
list(iter(t))

[1, 2, 3]

## Porównanie

In [41]:
# czy jest to ta sama wartość?
a = 100000
b = 100000
a == b

True

In [42]:
# czy jest to ten sam obiekt?
a = 100000
b = 100000
a is b

False

In [50]:
# pułapki optymalizacji CPythona
a = 1
b = 1
a==b, a is b

True


In [51]:
# porównanie w przypadku klas
class T:
    pass
a = T()
b = T()
c = a
a==b,a==c

(False, True)

In [52]:
# porównanie w przypadku klas
class T:
    def __eq__(self,other):
        return True
a = T()
b = T()
a==b

True

Atrybuty

In [53]:
class Door:
    def __init__(self, number, status):
        self.number = number
        self.status = status

    def open(self):
        self.status = 'open'

    def close(self):
        self.status = 'closed'

In [54]:
door1 = Door(1, 'closed')
door2 = Door(1, 'closed')
print(id(door1),id(door2))
print(id(door1.__class__),id(door2.__class__))
'0xb67e144c'

11177776 11177744
11202096 11202096


'0xb67e144c'

In [55]:
class Door:
    colour = 'brown'

    def __init__(self, number, status):
        self.number = number
        self.status = status

    def open(self):
        self.status = 'open'

    def close(self):
        self.status = 'closed'

In [56]:
door1 = Door(1, 'closed')
door2 = Door(1, 'closed')

In [57]:
print(id(Door.colour),id(door1.colour),id(door2.colour))

79455104 79455104 79455104


In [58]:
Door.colour, door1.colour, door2.colour

('brown', 'brown', 'brown')

In [61]:
Door.__dict__

mappingproxy({'__dict__': <attribute '__dict__' of 'Door' objects>,
              '__doc__': None,
              '__init__': <function __main__.Door.__init__>,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'Door' objects>,
              'close': <function __main__.Door.close>,
              'colour': 'brown',
              'open': <function __main__.Door.open>})

In [62]:
door1.__dict__

{'number': 1, 'status': 'closed'}

In [63]:
door1.colour = "white"

In [64]:
Door.open

<function __main__.Door.open>

In [65]:
door1.open

<bound method Door.open of <__main__.Door object at 0x00AA8250>>

In [66]:
door1.__class__.__dict__['open'].__get__(door1)

<bound method Door.open of <__main__.Door object at 0x00AA8250>>

## Dziedziczenie i MRO

In [67]:
class SecurityDoor(Door):
    pass

In [68]:
sdoor = SecurityDoor(1, 'closed')

In [69]:
sdoor.__dict__

{'number': 1, 'status': 'closed'}

In [70]:
SecurityDoor.__dict__

mappingproxy({'__doc__': None, '__module__': '__main__'})

In [71]:
SecurityDoor.__bases__

(__main__.Door,)

In [72]:
sdoor.open

<bound method SecurityDoor.open of <__main__.SecurityDoor object at 0x00AA8630>>

In [73]:
sdoor.__class__.__dict__['open'].__get__(sdoor)

KeyError: 'open'

In [74]:
sdoor.__class__.__bases__[0].__dict__['open'].__get__(sdoor)

<bound method SecurityDoor.open of <__main__.SecurityDoor object at 0x00AA8630>>

In [79]:
class A:
    def foo(self):
        print("A")
    def moo(self):
        pass
class B1(A):
    def foo(self):
        print("B1")
    def moo(self):
        pass
class B2(A):
    def foo(self):
        print("B2")
    def moo(self):
        pass
class C(B1,B2):
    #def foo(self):
    #    print("C")
    pass
oc = C()
oc.foo()
print(C.__mro__)

B1
(<class '__main__.C'>, <class '__main__.B1'>, <class '__main__.B2'>, <class '__main__.A'>, <class 'object'>)


### MRO

In [76]:
class X: pass
class Y: pass
class A(X,Y): pass
class B(Y,X): pass
class C(A,B): pass
C.__mro__

# https://en.wikipedia.org/wiki/C3_linearization

TypeError: Cannot create a consistent method resolution
order (MRO) for bases Y, X

In [80]:
class F(): pass
class E(): pass
class D(): pass
class C(D,F): pass
class B(D,E): pass
class A(B,C): pass
print(A.__mro__) 

(<class '__main__.A'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.D'>, <class '__main__.E'>, <class '__main__.F'>, <class 'object'>)


## Wyjątki też są klasami!

In [81]:
5 + '5'

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [82]:
try:
    5 + '5'
except TypeError:
    print('no-no')

no-no


In [83]:
try:
    5 + '5'
except TypeError as e:
    print(type(e), e)

<class 'TypeError'> unsupported operand type(s) for +: 'int' and 'str'


In [84]:
try:
    5 + '5'
except (TypeError, KeyError) as e:
    print(type(e), e)

<class 'TypeError'> unsupported operand type(s) for +: 'int' and 'str'


In [85]:
try:
    dict()[3]
except TypeError as e:
    print(type(e), e)
except KeyError as e:
    print("it's only key error")

it's only key error


In [86]:
class WrongAnswerError(Exception):
    pass

try:
    raise WrongAnswerError('wrong wrong wrong!')
except WrongAnswerError:
    print('good good good')
finally:
    print('bye!')

good good good
bye!


In [87]:
class A:
    pass

raise A

TypeError: exceptions must derive from BaseException

In [88]:
BaseException.__bases__

(object,)

In [89]:
Exception.__bases__

(BaseException,)

In [90]:
TypeError.__bases__

(Exception,)

In [91]:
KeyboardInterrupt.__bases__

(BaseException,)